In [54]:
!pip install yfinance pandas
%pip install Bottleneck

     ---------------------------------------- 0.0/109.9 kB ? eta -:--:--
     ----------------------------------- -- 102.4/109.9 kB 5.8 MB/s eta 0:00:01
     -------------------------------------- 109.9/109.9 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## 52 week high

In [1]:
import yfinance as yf
import vectorbt as vbt
import pandas as pd
import numpy as np

# Define the stock list and index
stock_list = ['TCS.NS', 'ITC.NS', 'VBL.NS', "DMART.NS", "TATAMOTORS.NS"]  # Example stock listn
index_symbol = '^NSEI'

# Download historical data
def download_data(symbols, interval, period):
    data = {}
    for symbol in symbols:
        data[symbol] = yf.download(symbol, interval=interval, period=period)
    return data

# Download 4 years of daily and hourly data
daily_data = download_data(stock_list, interval='1d', period='4y')
hourly_data = download_data(stock_list, interval='1H', period='650d')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
import yfinance as yf
import vectorbt as vbt
import pandas as pd
import numpy as np

# # Define the stock list and index
# stock_list = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']  # Example stock list
# index_symbol = '^NSEI'

# # Download historical data
# def download_data(symbols, interval, period):
#     data = {}
#     for symbol in symbols:
#         data[symbol] = yf.download(symbol, interval=interval, period=period)
#     return data

# # Download 4 years of daily and hourly data
# daily_data = download_data(stock_list, interval='1d', period='4y')
# hourly_data = download_data(stock_list, interval='60m', period='2y')

# Define the strategy
def strategy(data):
    # Calculate 52-week high
    high_52_week = data['High'].rolling(window=52*5).max()  # Assuming 5 trading days per week
    # Find stocks within 1-3% of the 52-week high
    within_range = (data['Close'] >= (high_52_week * 0.97)) 
    # Calculate MACD
    macd = vbt.MACD.run(data['Close'], fast_window=7, slow_window=14)
    # Generate buy signals where MACD is above the signal line
    buy_signals = within_range & (macd.macd > macd.signal)
    # Define take profit and stop loss
    tp_sl = pd.DataFrame({
        'tp': data['Close'] * 1.01,
        'sl': data['Close'] * 0.995
    })
    return buy_signals, tp_sl

# Backtest the strategy
# Backtest the strategy
def backtest_strategy(data, buy_signals, tp_sl):
    # Define stop loss and take profit as fractions
    sl_pct = 0.3  # 0.5% stop loss
    tp_pct = 0.5   # 1% take profit

    # Run the backtest
    pf = vbt.Portfolio.from_signals(
        close=data['Close'],
        entries=buy_signals,
        exits=None,
        sl_stop=sl_pct,
        tp_stop=tp_pct,
        size=1,  # Assuming we buy 1 unit of the asset
        size_type='amount',  # Buying a specific amount of the asset
        freq='1H',  # The frequency of the data
        upon_opposite_entry='close'
    )
    return pf


# Run the strategy and backtest for each stock
results = {}
for symbol in stock_list:
    buy_signals, tp_sl = strategy(hourly_data[symbol])
    results[symbol] = backtest_strategy(hourly_data[symbol], buy_signals, tp_sl)

# Compare with buy-and-hold strategy of the ^NSEI index
index_data = yf.download(index_symbol, interval='1d', period='2y')
index_returns = index_data['Close'].pct_change()
benchmark = vbt.Portfolio.from_holding(index_returns)

# Plot equity graphs
for symbol, result in results.items():
    result.plot().show()
benchmark.plot().show()


In [ ]:
import yfinance as yf
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the stock list and initial parameters
stock_list = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']  # Example stock list
index_symbol = '^NSEI'
initial_cash = 200000
max_investment_per_stock = initial_cash * 0.18
take_profit_pct = 0.03
stop_loss_pct = 0.02
max_trades_per_day_per_stock = 3

# Function to download 1-minute interval data for the current day
def download_intraday_data(symbols, date):
    data = {}
    for symbol in symbols:
        data[symbol] = yf.download(symbol, interval='1m', start=date, end=date + pd.Timedelta(days=1))
    return data

# Function to determine if a stock is near its 52-week high
def is_near_52_week_high(data):
    high_52_week = data['High'].rolling(window=52*5*390).max()  # Assuming 5 trading days per week, 390 minutes per trading day
    return data['Close'] >= (high_52_week * 0.97) & (data['Close'] <= high_52_week)

# Function to calculate MACD and determine if the fast MACD is above the slow MACD
def macd_above(data):
    macd = vbt.MACD.run(data['Close'], fast_window=12, slow_window=26)
    return macd.macd > macd.signal

# Function to sort stocks by market cap
def sort_by_market_cap(symbols):
    market_caps = {}
    for symbol in symbols:
        stock_info = yf.Ticker(symbol).info
        market_caps[symbol] = stock_info.get('marketCap', 0)
    sorted_symbols = sorted(market_caps, key=market_caps.get, reverse=True)
    return sorted_symbols

# Function to simulate the trading strategy
def simulate_trading():
    total_profit = 0
    cash = initial_cash
    for day in range(number_of_days_to_simulate):
        # Download data and apply strategy at 10 AM
        intraday_data = download_intraday_data(stock_list)
        near_52_week_high_stocks = [symbol for symbol in stock_list if is_near_52_week_high(intraday_data[symbol])]
        sorted_stocks = sort_by_market_cap(near_52_week_high_stocks)
        selected_stocks = sorted_stocks[len(sorted_stocks)//2 - 2:len(sorted_stocks)//2 + 3]  # Pick middle 5 stocks
        
        # Initialize portfolio and simulate trades
        portfolio = vbt.Portfolio(start_cash=cash)
        for symbol in selected_stocks:
            stock_data = intraday_data[symbol]
            if macd_above(stock_data):
                entries = np.zeros(len(stock_data), dtype=bool)
                entries[0] = True  # Buy at the first minute after 10 AM
                exits = np.zeros(len(stock_data), dtype=bool)
                # Set exits based on take profit and stop loss
                # ... (This part of the code needs to be implemented)
                
                # Simulate trades with vectorbt
                portfolio.from_signals(stock_data['Close'], entries, exits,
                                       fixed_cash_amount=max_investment_per_stock,
                                       take_profit=take_profit_pct,
                                       stop_loss=stop_loss_pct,
                                       max_trades=max_trades_per_day_per_stock)
        
        # Update cash and total profit for the next day
        cash += portfolio.total_profit()
        total_profit += portfolio.total_profit()
    
    return total_profit
# Download the ^NSEI index data
index_data = yf.download(index_symbol, interval='1d', period='1y')['Close']
index_returns = index_data.pct_change().fillna(0)

# Run the simulation for a specific date
date = pd.Timestamp('2023-01-03')  # Example date
simulate_trading(date)

# Assuming 'strategy_returns' is a Series of the strategy's daily returns
# Compare the strategy's performance with the ^NSEI index
strategy_cumulative_returns = (1 + strategy_returns).cumprod() - 1
index_cumulative_returns = (1 + index_returns).cumprod() - 1

# Plot the equity curve
plt.figure(figsize=(14, 7))
plt.plot(strategy_cumulative_returns, label='Strategy Equity Curve')
plt.plot(index_cumulative_returns, label='^NSEI Index Equity Curve')
plt.title('Equity Curve Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()


In [ ]:
import yfinance as yf
import requests
import pandas as pd
import vectorbt as vbt
from datetime import datetime, timedelta

# Your Polygon.io API key
api_key = 'YOUR_API_KEY'

# Define the stock list and initial parameters
stock_list = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']  # Example stock list
initial_cash = 200000
max_investment_per_stock = initial_cash * 0.18
take_profit_pct = 0.03
stop_loss_pct = 0.02
simulation_days = 90  # Number of days to simulate

def download_data_and_calculate_52_week_high(stock_list):
    stock_data = {}
    for symbol in stock_list:
        df = yf.download(symbol, period='2y', interval='1d')
        df['52_high'] = df['Close'].rolling(window=252).max()  # 252 trading days in a year
        df['near_52_week_high']= df['Open'] >= ( df['52_high']  * 0.96) & (df['Open'] <= df['52_high'] * 1.01)
        stock_data[symbol] = df
        macd = vbt.MACD.run(df['Close'], fast_window=12, slow_window=26)
    return stock_data

# Function to fetch one-minute interval data from Polygon.io
def fetch_intraday_data(symbol, date, api_key):
    formatted_date = date.strftime('%Y-%m-%d')
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/minute/{formatted_date}/{formatted_date}?apiKey={api_key}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.rename(columns={'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v': 'Volume'}, inplace=True)
    return df


def is_open_within_5_percent_of_52_high(stock_data, date):
    # Ensure the date is in the correct format (pandas Timestamp)
    if isinstance(date, str):
        date = pd.to_datetime(date)
    
    # Check if the date is within the DataFrame index
    if date not in stock_data.index:
        raise ValueError(f"The date {date} is not in the stock data index.")
    
    # Get the opening price and the 52-week high for the given date
    opening_price = stock_data.loc[date, 'Open']
    high_52_week = stock_data.loc[date, '52_high']
    
    # Check if the opening price is within 5% of the 52-week high
    is_within_5_percent = opening_price >= (high_52_week * 0.95)
    
    return is_within_5_percent

# Function to determine if a stock is near its 52-week high
def is_near_52_week_high(data):
    # high_52_week = data['High'].rolling(window=52*5*390).max()  # Assuming 5 trading days per week, 390 minutes per trading day
    return data['Close'] >= ( data['52_high']  * 0.96) & (data['Close'] <= data['52_high'] * 1.01)

# Function to calculate MACD and determine if the fast MACD is above the slow MACD
def macd_above(data):
    macd = vbt.MACD.run(data['Close'], fast_window=12, slow_window=26)
    return macd.macd > macd.signal

# Function to sort stocks by market cap
def sort_by_market_cap(symbols):
    market_caps = {}
    for symbol in symbols:
        stock_info = yf.Ticker(symbol).info
        market_caps[symbol] = stock_info.get('marketCap', 0)
    sorted_symbols = sorted(market_caps, key=market_caps.get, reverse=True)
    return sorted_symbols

# Function to simulate the trading strategy for a given day
def simulate_trading_for_day(date, stock_data, high_52_week):
    if isinstance(date, str):
        date = pd.to_datetime(date)
    cash = initial_cash
    portfolio = vbt.Portfolio(start_cash=cash)
    
    # Filter stocks near 52-week high
    stock_list = [symbol for symbol in stock_data.keys() if stock_data[symbol].loc[date,'near_52_week_high']]
    symbols = sort_by_market_cap(stock_list)
    
    
    for symbol in stock_list:
        if daily_data[symbol]['Close'].iloc[-1] >= high_52_week[symbol].iloc[-1] * 0.97:
            selected_stocks.append(symbol)
    
    # Download one-minute data and execute trades
    for symbol in selected_stocks:
        
        intraday_data = fetch_intraday_data(symbol, date, api_key)
        # Define entry condition: stock price reaches within 3% of the 52-week high
        entries = intraday_data['Close'] >= high_52_week[symbol].iloc[-1] * 0.97
        # Define exits based on stop loss and take profit
        # Simulate trades with vectorbt
        # ...

    return portfolio

# Simulate trading over the 90-day period
start_date = datetime.now() - timedelta(days=simulation_days)
for single_date in pd.date_range(start_date, periods=simulation_days):
    portfolio = simulate_trading_for_day(single_date, daily_data, high_52_week)
    # Update cash for the next day
    initial_cash += portfolio.total_profit()

# Plot the equity curve
portfolio.plot().show()


# LumiBot

In [1]:
%pip install vectorbt


     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     -- ------------------------------------- 0.2/2.5 MB 4.6 MB/s eta 0:00:01
     -------- ------------------------------- 0.5/2.5 MB 8.4 MB/s eta 0:00:01
     ------------- -------------------------- 0.8/2.5 MB 8.6 MB/s eta 0:00:01
     ---------------------- ----------------- 1.4/2.5 MB 8.9 MB/s eta 0:00:01
     ---------------------------- ----------- 1.8/2.5 MB 8.8 MB/s eta 0:00:01
     -------------------------------------- - 2.4/2.5 MB 9.0 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 9.3 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/23.2 MB ? eta -:--:--
      --------------------------------------- 0.6/23.2 MB 17.5 MB/s eta 0:00:02
     - -------------------------------------- 1.0/23.2 MB 12.2 MB/s eta 0:00:02
     -- ------------------------------------- 1.4/23.2 MB 11.1 MB/s eta 0

In [1]:
import yfinance as yf
import vectorbt as vbt
import pandas as pd
import numpy as np
import pandas_ta as ta
import datetime 

from lumibot.backtesting import YahooDataBacktesting
from lumibot.brokers import Alpaca
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader


from lumibot.backtesting import BacktestingBroker, PolygonDataBacktesting
from lumibot.strategies import Strategy
from lumibot.traders import Trader
from sklearn.preprocessing import MinMaxScaler
from lumibot.backtesting import PandasDataBacktesting, BacktestingBroker


In [2]:
stock_list = [x+".NS" for x in pd.read_csv("ind_nifty100list.csv")["Symbol"]]
# stock_data = {}
# for stock in stock_list:    
#     data = yf.download(stock, period="60d", interval= "5m")
#     stock_data[stock] = data


In [3]:
def download_yearly_data(stock_list):
    stock_data = {}
    for symbol in stock_list:
      df = yf.download(symbol, period='2y', interval='1d')
      df['52_high'] = df['Close'].shift(10).rolling(window=252).max()  # 252 trading days in a year
      macd =  df.ta.macd()
      df["filter"] = (macd.iloc[:,0] > macd.iloc[:,-1]) & ( df['Open'] >= ( df['52_high']  * 1.01) )     
      scaler = MinMaxScaler()
      df['Volume'] = df['Volume'].shift(1) 
      df = df.iloc[263:]

      try:
        df["Volume"] = scaler.fit_transform(df["Volume"].values.reshape(-1, 1))
      except Exception as e:
         print(symbol, e)
         continue
         
      df.rename(columns={"Open": "open", "High": "high", "Low": "low", 'Close':"temp", "Adj Close": "close", "Volume":  "volume"}, inplace=True)
      stock_data[symbol] = df
      stock_data[symbol].drop(columns=["temp"], inplace=True)
      stock_data[symbol].index.name = "datetime"
      # break
    return stock_data

In [4]:
stock_data= {}

In [5]:
stock_data = download_yearly_data(stock_list=stock_list)

[*********************100%%**********************]  1 of 1 completed
C:\Users\Shubham\AppData\Local\Temp\ipykernel_3928\2818112028.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Volume"] = scaler.fit_transform(df["Volume"].values.reshape(-1, 1))
C:\Users\Shubham\AppData\Local\Temp\ipykernel_3928\2818112028.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Open": "open", "High": "high", "Low": "low", 'Close':"temp", "Adj Close": "close", "Volume":  "volume"}, inplace=True)
C:\Users\Shubham\AppData\Local\Temp\ipykernel_3928\28181120

ADANIENSOL.NS Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.



C:\Users\Shubham\AppData\Local\Temp\ipykernel_3928\2818112028.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Volume"] = scaler.fit_transform(df["Volume"].values.reshape(-1, 1))
C:\Users\Shubham\AppData\Local\Temp\ipykernel_3928\2818112028.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Open": "open", "High": "high", "Low": "low", 'Close':"temp", "Adj Close": "close", "Volume":  "volume"}, inplace=True)
C:\Users\Shubham\AppData\Local\Temp\ipykernel_3928\2818112028.py:20: SettingWithCopyWarning: 
A value is trying to be set on a 

In [6]:
print(len(stock_data))

100


In [7]:
backup_stock_data ={}
for k in stock_data:
    backup_stock_data[k] = stock_data[k].copy()

In [8]:
# for k in backup_stock_data:
#     stock_data[k] = backup_stock_data[k].copy()

In [9]:
# ## preprocess volume in the stock data
# for symbol in stock_data.keys():
#     scaler = MinMaxScaler()
#     stock_data[symbol]["Volume"] = scaler.fit_transform(stock_data[symbol]["Volume"].values.reshape(-1, 1))
#     stock_data[symbol].columns = ['open', 'high', 'low', 'temp', "close", 'volume']
#     stock_data[symbol].drop(columns=["temp"], inplace=True)
#     stock_data[symbol].index.name = "datetime"
    
# # stock_data


In [10]:
## Prepare data for lumibot
from lumibot.entities import Asset ,Data
pandas_data = {
}
for symbol in stock_data.keys():
    
    asset = Asset(
    symbol=symbol,
    asset_type="stock",
            )
    data = Data(
    asset,
    stock_data[symbol],
    trading_hours_start=datetime.time(9, 30),
    trading_hours_end=datetime.time(15, 10),
    timestep="day",
#     timezone="Asia/Kolkata",
    )
    pandas_data[asset] = data



In [11]:
from lumibot.strategies import Strategy

class MyStrategy(Strategy):

    def initialize(self, parameters=None):
        self.stock_data = stock_data
        self.sleeptime = "24H"
        self.count = 0
        super().initialize(parameters)
    def on_trading_iteration(self):
     # get all sumbols where filter is True
        
        today = pd.Timestamp(self.get_datetime())
        print(today)
        symbol_volume = [(symbol,data['volume']) for symbol, data in self.stock_data.items() if today in data.index and data.loc[today,'filter']]
        symbol_volume.sort(key=lambda x: x[1], reverse=True)
        reamaining_cash = self.cash
        total_positions = min(len(symbol_volume) , 4)
        if total_positions == 0:
            pass
        for i in range(total_positions):
            last_price = self.get_last_price(symbol_volume[i][0])
            if reamaining_cash//total_positions >= last_price:
                quantity = reamaining_cash // self.get_last_price(symbol_volume[i][0])
                order = self.create_order(symbol_volume[i][0], quantity, "buy", take_profit_price=last_price*1.015, stop_loss_price=last_price*0.9925)
                self.submit_order(order)

        # 


In [12]:
# Pick the date range you want to backtest
backtesting_start = pandas_data[asset].datetime_start
backtesting_end = pandas_data[asset].datetime_end

# Run the backtesting
trader = Trader(backtest=True, debug=True)
data_source = PandasDataBacktesting(
    pandas_data=pandas_data,
    datetime_start=backtesting_start,
    datetime_end=backtesting_end,
)
broker = BacktestingBroker(data_source)
strat = MyStrategy(
    broker=broker,
    budget=100000,
    sleeptime="1440M"
)
trader.add_strategy(strat)
trader.run_all()

2024-02-12 18:49:28,225: root: INFO: MyStrategy : Executing the initialize lifecycle method
2024-02-12 18:49:30,089: root: INFO: MyStrategy : Executing the before_market_opens lifecycle method
2024-02-12 18:49:30,089: root: INFO: MyStrategy : Executing the before_starting_trading lifecycle method
2024-02-12 18:49:30,105: root: INFO: MyStrategy : Executing the after_market_closes lifecycle method
2024-02-12 18:49:30,105: root: INFO: MyStrategy : Executing the before_market_opens lifecycle method
2024-02-12 18:49:30,105: root: INFO: MyStrategy : Executing the before_starting_trading lifecycle method
2024-02-12 18:49:30,105: root: INFO: MyStrategy : Executing the after_market_closes lifecycle method
2024-02-12 18:49:30,105: root: INFO: MyStrategy : Executing the before_market_opens lifecycle method
2024-02-12 18:49:30,105: root: INFO: MyStrategy : Executing the before_starting_trading lifecycle method
2024-02-12 18:49:30,123: root: INFO: MyStrategy : Executing the after_market_closes life

In [66]:
ts=pd.Timestamp('2022-02-11 00:00:00+0530', tz='Asia/Kolkata')
(stock_data['ABB.NS'].loc[ts])


open            2224.0
high       2249.899902
low        2170.050049
close      2162.147217
volume        0.057683
52_high            NaN
filter           False
Name: 2022-02-11 00:00:00+05:30, dtype: object

# vectorbt

In [1]:
import numpy as np
from numba import njit
import vectorbt as vbt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import bottleneck as bn

In [2]:
def sort_by_market_cap(symbols):
    market_caps = {}
    for symbol in symbols:
        stock_info = yf.Ticker(symbol).info
        market_caps[symbol] = stock_info.get('marketCap', 0)
    sorted_symbols = sorted(market_caps, key=market_caps.get, reverse=True)
    return sorted_symbols

In [3]:
@njit
def make_buy_before_sell(arr):
    # Initialize a variable to keep track of the last non-zero value encountered
    last_non_zero = 0
    last_idx = 0
    n= len(arr)
    # Iterate through the array
    for i in range(n):
        # If the current element is 1 (buy)
        if arr[i] == 1:
            # If the last non-zero value was also 1, set the current element to 0
            if last_non_zero == 1:
                arr[i] = 0
            last_non_zero = 1  # Update the last non-zero value
            last_idx = i  # Update the last index
        # If the current element is -1 (sell)
        elif arr[i] == -1:
            # If the last non-zero value was not 1 (no buy before), set the current element to 0
            if last_non_zero != 1:
                arr[i] = 0
            last_non_zero = -1  # Update the last non-zero value
        # If the current element is 0, do nothing and continue
    # if last non zero value is 1, set the last element to 0
    if last_non_zero == 1:
        arr[last_idx] = 0
    return arr

In [4]:
@njit
def shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

In [5]:
stock_list = [sym+".NS" for sym in pd.read_csv('ind_nifty100list.csv').Symbol]
stock_list = sort_by_market_cap(stock_list)
stockss=stock_list[:70]
ohlcv = vbt.YFData.download(stockss, period = '2y', interval='1d')

# type(ohlcv['Open'])
# stock_list


In [6]:
def zscore_signal(close, length, lower_threshold=2.0, upper_threshold=2.0):
    """
    Calculates the z-score of the difference between the current close and the
    previous close.
    """
    print(close.shape)
    zscore = vbt.IndicatorFactory.from_pandas_ta("zscore").run(close, length=length).zs
    exits = np.where(zscore > upper_threshold, -1, 0)
    trend = np.where(zscore < lower_threshold, 1, exits)
    trend = shift(trend,1,0)
    # trend = make_buy_before_sell(trend)
    trend = np.apply_along_axis(make_buy_before_sell, 0, trend)
    return trend

In [13]:
zsignal = vbt.IndicatorFactory(
         class_name="zscore",
         short_name="ma",
         input_names=["close"],
         param_names=["length","lower_threshold","upper_threshold"],
         output_names=["signal"]
         ).from_apply_func(
             zscore_signal,
             length=20,
             lower_threshold=1.5,
             upper_threshold=-1.5
         )


In [27]:
start_date = '2016-01-01'
end_date = '2018-12-31'

ohlcv = vbt.YFData.download("^NSEI", start=start_date, end=end_date, interval='1d')

## madc 1

### backup

In [177]:
def madc_signal(close, high, low, dc_len, ema_len):
    # vbt.IndicatorFactory.from_pandas_ta("donchian", high= high, low=low,  lower_length = dc_len, upper_length = dc_len)
    mid= pd.DataFrame()
    ema = pd.DataFrame()
    for col in high.columns:
        h = high[col].rolling(dc_len).max()
        l = low[col].rolling(dc_len).min()
        mid[col]= (h+l)/2
        # mid[col] = mid[col].shift(1)
        ema[col]= ta.ema(close[col],length=ema_len)
        # ema[col] = ema[col].shift(1)
    
    ema1 = vbt.MA.run(close, ema_len).ma.to_numpy()
    sma50 = vbt.MA.run(close, 50).ma.to_numpy()
    base_condtion = np.where(close > sma50, 1, 0)
    mid = mid.to_numpy()
    ema = ema.to_numpy()
    entries = np.where((ema > mid) & (base_condtion > 0), 1, 0  )
    trends = np.where((ema < mid ) & (base_condtion > 0), -1, entries)
    # mid = np.where(base_condtion == 1, mid, 0)
    # entries = np.where(ema.ma_crossed_above(mid) > 0, sma50, 0  )
    # exits = np.where(ema.ma_crossed_below(mid) > 0 , sma50, 0)   
    # mid = mid.to_numpy()
    # ema = ema.to_numpy()
    # entries = np.where(ema > mid, 1, 0)
    # signal = np.where(ema < mid, -1, entries)
    # trend = shift(signal,1,0)
    # trend = make_buy_before_sell(trend)
    # trend = np.apply_along_axis(make_buy_before_sell, 0, trend)
    return trends , mid, ema, sma50 , ema1


### true

In [7]:
def generate_params(ema_length, dc_len, gap):
    ema_l = []
    dc_l = []
    for e in ema_length:
        for d in dc_len:
            if d -e >= gap:
                ema_l.append(e)
                dc_l.append(d)
    return ema_l, dc_l

In [8]:
import numba
from numba import jit

In [28]:
def madc_signal(close, high, low, dc_len, ema_len):
    # vbt.IndicatorFactory.from_pandas_ta("donchian", high= high, low=low,  lower_length = dc_len, upper_length = dc_len)
    mid= pd.DataFrame()
    ema = pd.DataFrame()
    for col in high.columns:
        h = high[col].rolling(dc_len).max(engine='numba')
        l = low[col].rolling(dc_len).min(engine='numba')
        mid[col]= (h+l)/2
        # mid[col] = mid[col].shift(3)
        ema[col]= ta.ema(close[col],length=ema_len)

    sma50 = vbt.MA.run(close, 50).ma.to_numpy()
    base_condtion = np.where(close > sma50, 1, 0)
    mid = mid.to_numpy()
    ema = ema.to_numpy()
    entries = np.where((ema > mid) & (base_condtion > 0), 1, 0  )
    trends = np.where((ema < mid ), -1, entries)
    return trends , mid, ema, sma50 

madc = vbt.IndicatorFactory(
         class_name="madc",
         short_name="ma",
         input_names=["close", "low", "high"],
         param_names=["dc_len", "ema_length"],
         output_names=["trends", "mid", "ema", "sma50",]
         ).from_apply_func(
             madc_signal,
             dc_len=7,
             ema_length=4,
            keep_pd=True,
            param_product = True
         )
# signal = zsignal.run(ohlcv.get("Close"), length=70,lower_threshold=-2.5, upper_threshold=3.3,param_product=True)
ema_length=np.arange(3,28,2)
dc_len=np.arange(8,30,1)
# ema_length, dc_len = generate_params(ema_length, dc_len, 3)

# long_dec_len=[18,19 ,20, 21,22,23,24]
# long_ema_length=[ 9, 10, 12, 13, 14, 15, 16]
signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_length, dc_len=dc_len)
entries = signal.trends_crossed_above(0)
exits = signal.trends_crossed_below(0)
# base_condtion = np.where(ohlcv.get("Close") > signal.sma50, 1, 0)
# ema = vbt.MA.run(ohlcv.get("Close"), 4)
# entries = np.where(ema.ma_crossed_above(signal.mid) > 0,base_condtion, 0 )
# exits = np.where(ema.ma_crossed_below(signal.mid) > 0 , base_condtion, 0)  
pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1d', init_cash=100000,cash_sharing=False)
pf.stats()



C:\Users\Shubham\AppData\Local\Temp\ipykernel_6056\356176036.py:48: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x0000023FD65204C0>. Pass column to select a single column/group.



Start                           2016-01-03 18:30:00+00:00
End                             2018-12-27 18:30:00+00:00
Period                                  736 days 00:00:00
Start Value                                      100000.0
End Value                                   120335.062194
Total Return [%]                                20.335062
Benchmark Return [%]                            39.384964
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                 6.645599
Max Drawdown Duration         199 days 09:59:09.650349650
Total Trades                                    25.583916
Total Closed Trades                             24.814685
Total Open Trades                                0.769231
Open Trade PnL                                 177.846612
Win Rate [%]                                    62.336503
Best Trade [%]                                   7.449108
Worst Trade [%

In [179]:
madc = vbt.IndicatorFactory(
         class_name="madc",
         short_name="ma",
         input_names=["close", "low", "high"],
         param_names=["dc_len", "ema_length"],
         output_names=["trends", "mid", "ema", "sma50",]
         ).from_apply_func(
             madc_signal,
             dc_len=7,
             ema_length=4,
            keep_pd=True,
            param_product = True
         )

## madc2

In [15]:
def madc_signal(close, high, low, dc_len, ema_len):
    # vbt.IndicatorFactory.from_pandas_ta("donchian", high= high, low=low,  lower_length = dc_len, upper_length = dc_len)
    mid= pd.DataFrame()
    for col in high.columns:
        h = high[col].rolling(dc_len).max()
        l = low[col].rolling(dc_len).min()
        mid[col]= (h+l)/2
        mid[col] = mid[col].shift(1)
    ema = vbt.MA.run(close, ema_len)
    sma50 = vbt.MA.run(close, 50).ma.to_numpy()
    base_condtion = np.where(close > sma50, 1, 0)
    mid = mid.to_numpy()
    # entries = np.where(ema.ma_crossed_above(mid) > 0, sma50, 0  )
    # exits = np.where(ema.ma_crossed_below(mid) > 0 , sma50, 0)   
    # mid = mid.to_numpy()
    ema = ema.ma.to_numpy()
    # entries = np.where(ema > mid, 1, 0)
    # signal = np.where(ema < mid, -1, entries)
    # trend = shift(signal,1,0)
    # trend = make_buy_before_sell(trend)
    # trend = np.apply_along_axis(make_buy_before_sell, 0, trend)
    return mid ,ema, sma50


In [16]:
madc = vbt.IndicatorFactory(
         class_name="madc",
         short_name="ma",
         input_names=["close", "low", "high"],
         param_names=["dc_len", "ema_length"],
         output_names=["mid", "ema","sma50"]
         ).from_apply_func(
             madc_signal,
             dc_len=7,
             ema_length=4,
            keep_pd=True
         )

## madc 3

In [18]:
def madc_signal(close, high, low, dc_len, ema_len):
    # vbt.IndicatorFactory.from_pandas_ta("donchian", high= high, low=low,  lower_length = dc_len, upper_length = dc_len)
    mid= pd.DataFrame()
    for col in high.columns:
        h = high[col].rolling(dc_len).max()
        l = low[col].rolling(dc_len).min()
        mid[col]= (h+l)/2
        mid[col] = mid[col].shift(1)
    ema = vbt.MA.run(close, ema_len)
    sma50 = vbt.MA.run(close, 50).ma.to_numpy()
    base_condtion = np.where(close > sma50, 1, 0)
    mid = mid.to_numpy()
    entries = np.where(ema.ma_crossed_above(mid) > 0, sma50, 0  )
    exits = np.where(ema.ma_crossed_below(mid) > 0 , sma50, 0)   
    # mid = mid.to_numpy()
    # ema = ema.to_numpy()
    # entries = np.where(ema > mid, 1, 0)
    # signal = np.where(ema < mid, -1, entries)
    # trend = shift(signal,1,0)
    # trend = make_buy_before_sell(trend)
    # trend = np.apply_along_axis(make_buy_before_sell, 0, trend)
    return entries ,exits


madc = vbt.IndicatorFactory(
         class_name="madc",
         short_name="ma",
         input_names=["close", "low", "high"],
         param_names=["dc_len", "ema_length"],
         output_names=["entries", "exits"],
         ).from_apply_func(
             madc_signal,
             dc_len=7,
             ema_length=4,
            keep_pd=True,
            param_product = True
         )

# signal = zsignal.run(ohlcv.get("Close"), length=70,lower_threshold=-2.5, upper_threshold=3.3,param_product=True)
ema_length=[2, 3,  7, 8, 9, 10, 12, 15, 16 ,17,15, 16 ,17,15, 16 ,17,18]
dc_len=[9,10,11,12,15, 16, 17, 20, 21,22,23,24,24,24,25,25,25,26,26,26,26]
# long_dec_len=[18,19 ,20, 21,22,23,24]
# long_ema_length=[ 9, 10, 12, 13, 14, 15, 16]
signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_length, dc_len=dc_len)
entries = signal.entries
exits = signal.exits 
# base_condtion = np.where(ohlcv.get("Close") > signal.sma50, 1, 0)
# ema = vbt.MA.run(ohlcv.get("Close"), 4)
# entries = np.where(ema.ma_crossed_above(signal.mid) > 0,base_condtion, 0 )
# exits = np.where(ema.ma_crossed_below(signal.mid) > 0 , base_condtion, 0)  
pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1d', init_cash=100000,cash_sharing=False)
pf.stats()


C:\Users\Shubham\AppData\Local\Temp\ipykernel_6056\3425905635.py:52: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x0000023FD65204C0>. Pass column to select a single column/group.



Start                           2019-01-01 18:30:00+00:00
End                             2022-12-29 18:30:00+00:00
Period                                  987 days 00:00:00
Start Value                                      100000.0
End Value                                    147508.20971
Total Return [%]                                 47.50821
Benchmark Return [%]                            67.758173
Max Gross Exposure [%]                              100.0
Total Fees Paid                                       0.0
Max Drawdown [%]                                23.150769
Max Drawdown Duration         307 days 10:33:16.638655464
Total Trades                                    44.445378
Total Closed Trades                              44.35014
Total Open Trades                                0.095238
Open Trade PnL                                 -13.994085
Win Rate [%]                                    52.320177
Best Trade [%]                                  16.683324
Worst Trade [%

In [12]:
benchpf = vbt.Portfolio.from_holding(ohlcv.get('Close'))
ret = benchpf.total_return() 
print(benchpf.stats())
bench_ret = ret.copy()

c:\Users\Shubham\Documents\stock-analyzer\stock-analyzer\.venv\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\Shubham\Documents\stock-analyzer\stock-analyzer\.venv\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\Shubham\Documents\stock-analyzer\stock-analyzer\.venv\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\Shubham\Documents\stock-analyzer\stock-analyzer\.venv\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2022-02-15 18:30:00+00:00
End                           2024-02-15 18:30:00+00:00
Period                                              495
Start Value                                       100.0
End Value                                    127.239613
Total Return [%]                              27.239613
Benchmark Return [%]                          27.239613
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              15.287427
Max Drawdown Duration                             133.0
Total Trades                                          1
Total Closed Trades                                   0
Total Open Trades                                     1
Open Trade PnL                                27.239613
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 

AttributeError: module 'vectorbt' has no attribute 'StatsBuilderMixin'

In [46]:
ret = pf.stats(['benchmark_return','total_return'])
print(ret.values)

[39.38496352 20.33506219]


C:\Users\Shubham\AppData\Local\Temp\ipykernel_6056\1992114435.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x0000023FD65204C0>. Pass column to select a single column/group.



In [161]:
# signal = zsignal.run(ohlcv.get("Close"), length=70,lower_threshold=-2.5, upper_threshold=3.3,param_product=True)
ema_length=[2, 3,  7, 8, 9, 10, 12, 15]
dc_len=[9,10,11,12,15, 16, 17, 20, 21,22,23,24]
# long_dec_len=[18,19 ,20, 21,22,23,24]
# long_ema_length=[ 9, 10, 12, 13, 14, 15, 16]
signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_length, dc_len=dc_len)
entries = signal.trends_crossed_above(0)
exits = signal.trends_crossed_below(0)
# base_condtion = np.where(ohlcv.get("Close") > signal.sma50, 1, 0)
# ema = vbt.MA.run(ohlcv.get("Close"), 4)
# entries = np.where(ema.ma_crossed_above(signal.mid) > 0,base_condtion, 0 )
# exits = np.where(ema.ma_crossed_below(signal.mid) > 0 , base_condtion, 0)  
pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1d', init_cash=100000,cash_sharing=False)
pf.stats()


C:\Users\Shubham\AppData\Local\Temp\ipykernel_11808\3885483715.py:14: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x000001995165C4C0>. Pass column to select a single column/group.



Start                          2020-02-16 18:30:00+00:00
End                            2024-02-14 18:30:00+00:00
Period                                 991 days 00:00:00
Start Value                                     100000.0
End Value                                  279893.497558
Total Return [%]                              179.893498
Benchmark Return [%]                           247.94802
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               25.037493
Max Drawdown Duration                  328 days 17:43:30
Total Trades                                   37.095833
Total Closed Trades                            36.421667
Total Open Trades                               0.674167
Open Trade PnL                              16107.479808
Win Rate [%]                                   57.973883
Best Trade [%]                                 30.415331
Worst Trade [%]                

In [29]:
returns = pf.total_return()
#filter result 
# filtered_returns = returns.loc[returns > returns.index.get_level_values('symbol').map(bench_ret)]
# returns.loc[returns < returns.index.get_level_values('symbol').map(bench_ret)] = 0
# returns = returns/ returns.index.get_level_values('symbol').map(bench_ret)
# print(len(filtered_returns), len(returns))
# returns = returns.groupby(level=['ma_dc_len','ma_ema_length']).mean()
print(returns.max())
print(returns.idxmax())

0.3799794109878052
(29, 5)


In [ ]:
returns

In [ ]:
fig = returns.vbt.volume(
    x_level= "ma_dc_len",
    y_level="ma_ema_length",
    z_level="symbol"
)
fig.show()

In [30]:
fig = returns.vbt.heatmap(
    x_level= "ma_dc_len",
    y_level="ma_ema_length",
  
)
fig.show()

In [ ]:
pf[(9, 11, 'TECHM.NS')].plot()

In [205]:
import plotly.graph_objs as go
key = (21, 10, 'BEL.NS')
fig = pf[key].plot_orders()
trace_ema = go.Scatter(
    x= signal[key].ema.index,
    y=signal[key].ema.values,
    mode='lines',
    name='EMA'
)

trace_mid = go.Scatter(
    x= signal[key].mid.index,
    y=signal[key].mid.values,
    mode='lines',
    name='Mid'
)
trace_sma50 = go.Scatter(
    x= signal[key].sma50.index,
    y=signal[key].sma50.values,
    mode='lines',
    name='sma50'
)
fig.add_trace(trace_ema)    
fig.add_trace(trace_mid)
fig.add_trace(trace_sma50)  
# fig.add_trace(trace_ema1)
fig.update_layout(title=str(key))
fig.show()

## Optimize and forward test

In [11]:
### utility

def plot_orders(pf, signal,key ):
    import plotly.graph_objs as go
    
    fig = pf.plot_orders()
    trace_ema = go.Scatter(
        x= signal.ema.index,
        y=signal.ema.values,
        mode='lines',
        name='EMA'
    )

    trace_mid = go.Scatter(
        x= signal.mid.index,
        y=signal.mid.values,
        mode='lines',
        name='Mid'
    )
    trace_sma50 = go.Scatter(
        x= signal.sma50.index,
        y=signal.sma50.values,
        mode='lines',
        name='sma50'
    )
    fig.add_trace(trace_ema)    
    fig.add_trace(trace_mid)
    fig.add_trace(trace_sma50)  
    # fig.add_trace(trace_ema1)
    fig.update_layout(title=str(key))
    fig.show()

In [29]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
def get_last_date(date :str) :
    date = datetime.strptime(date, '%Y-%m-%d')

    # Get the last date of the next month
    next_month = date + relativedelta(months=2)
    last_day_next_month = next_month.replace(day=1) - timedelta(days=1)

    last_day_next_month_string = last_day_next_month.strftime('%Y-%m-%d')
    test_start = last_day_next_month - timedelta(days=10)
    test_end = (last_day_next_month + relativedelta(months=3)).replace(day=1) -timedelta(days=1)
    test_start_string = test_start.strftime('%Y-%m-%d')
    test_end_string = test_end.strftime('%Y-%m-%d')
    next_month_string = (date + relativedelta(months=1)).replace(day=1).strftime('%Y-%m-%d')
    return last_day_next_month_string, test_start_string, test_end_string, next_month_string

get_last_date('2020-01-01')


('2020-02-29', '2020-02-19', '2020-04-30', '2020-02-01')

In [66]:
start_date = '2023-11-01'
end_date = '2023-12-31'

ohlcv = vbt.YFData.download(stockss, start=start_date, end=end_date, interval='1h')


ema_length=np.arange(3,30,2)
dc_len=np.arange(8,30,1)
signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_length, dc_len=dc_len)
entries = signal.trends_crossed_above(0)
exits = signal.trends_crossed_below(0)

pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
ret = pf.total_return()

            







In [67]:

best_val = {}
grid_pos = (-2,-1,0,1,2)
for st in stockss:
    cur_best = 0
    cur_params = (dc_len[0],ema_length[0])
    for i, dc_l in enumerate(dc_len):
        for j, ema_l in enumerate(ema_length):
            if i- min(grid_pos)<=0 or j- min(grid_pos)<=0 or i+max(grid_pos) >= len(dc_len) or j + max(grid_pos) >= len(ema_length):
                continue
            sm = 0
            for dx in grid_pos:
                for dy in grid_pos:
                    sm += ret[(dc_len[i+dx], ema_length[j+dy], st)]
            if sm > cur_best:
                cur_best = sm
                cur_params = (dc_l,ema_l)
    best_val[st] = cur_params
print(best_val)

{'RELIANCE.NS': (17, 21), 'TCS.NS': (23, 19), 'HDFCBANK.NS': (17, 7), 'ICICIBANK.NS': (27, 7), 'INFY.NS': (11, 17), 'SBIN.NS': (27, 7), 'BHARTIARTL.NS': (27, 9), 'LICI.NS': (27, 7), 'HINDUNILVR.NS': (25, 11), 'ITC.NS': (10, 25), 'LT.NS': (19, 15), 'HCLTECH.NS': (19, 17), 'BAJFINANCE.NS': (27, 5), 'ADANIENT.NS': (12, 7), 'SUNPHARMA.NS': (27, 15), 'MARUTI.NS': (19, 15), 'ONGC.NS': (18, 13), 'KOTAKBANK.NS': (27, 9), 'TATAMOTORS.NS': (23, 11), 'TATAMTRDVR.NS': (24, 15), 'AXISBANK.NS': (27, 13), 'NTPC.NS': (18, 7), 'TITAN.NS': (20, 17), 'ADANIGREEN.NS': (12, 7), 'COALINDIA.NS': (27, 25), 'ASIANPAINT.NS': (20, 21), 'ULTRACEMCO.NS': (25, 9), 'WIPRO.NS': (11, 17), 'ADANIPORTS.NS': (27, 7), 'IOC.NS': (17, 7), 'POWERGRID.NS': (24, 11), 'BAJAJFINSV.NS': (27, 9), 'DMART.NS': (22, 15), 'NESTLEIND.NS': (13, 7), 'BAJAJ-AUTO.NS': (27, 11), 'M&M.NS': (20, 11), 'DLF.NS': (26, 19), 'HAL.NS': (13, 7), 'JSWSTEEL.NS': (21, 11), 'VBL.NS': (26, 17), 'TATASTEEL.NS': (19, 7), 'LTIM.NS': (27, 21), 'SIEMENS.NS': 

In [16]:
# remove 'ADANIENSOL.NS' from stockks list
stockss.remove('ADANIENSOL.NS')

['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'INFY.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'LICI.NS', 'HINDUNILVR.NS', 'ITC.NS', 'LT.NS', 'HCLTECH.NS', 'BAJFINANCE.NS', 'ADANIENT.NS', 'SUNPHARMA.NS', 'MARUTI.NS', 'ONGC.NS', 'KOTAKBANK.NS', 'TATAMOTORS.NS', 'TATAMTRDVR.NS', 'AXISBANK.NS', 'NTPC.NS', 'TITAN.NS', 'ADANIGREEN.NS', 'COALINDIA.NS', 'ASIANPAINT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS', 'ADANIPORTS.NS', 'IOC.NS', 'POWERGRID.NS', 'BAJAJFINSV.NS', 'DMART.NS', 'NESTLEIND.NS', 'BAJAJ-AUTO.NS', 'M&M.NS', 'DLF.NS', 'HAL.NS', 'JSWSTEEL.NS', 'VBL.NS', 'TATASTEEL.NS', 'LTIM.NS', 'SIEMENS.NS', 'SBILIFE.NS', 'PNB.NS', 'TRENT.NS', 'GRASIM.NS', 'BANKBARODA.NS', 'BPCL.NS', 'PIDILITIND.NS', 'BEL.NS', 'ZOMATO.NS', 'HDFCLIFE.NS', 'TECHM.NS', 'GODREJCP.NS', 'TATAPOWER.NS', 'GAIL.NS', 'BRITANNIA.NS', 'INDIGO.NS', 'CIPLA.NS', 'INDUSINDBK.NS', 'AMBUJACEM.NS', 'HINDALCO.NS', 'ATGL.NS', 'TATACONSUM.NS', 'EICHERMOT.NS', 'CANBK.NS', 'DRREDDY.NS', 'TVSMOTOR.NS']


In [37]:

def optimize(pf, stockss, dc_len, ema_length):
    ret = pf.total_return() 
    best_val = {}
    best_opt_val={}
    grid_pos = (-2,-1,0,1,2)
    for st in stockss:
        cur_best = 0
        cur_params = (dc_len[0],ema_length[0])
        for i, dc_l in enumerate(dc_len):
            for j, ema_l in enumerate(ema_length):
                if i- min(grid_pos)<=0 or j- min(grid_pos)<=0 or i+max(grid_pos) >= len(dc_len) or j + max(grid_pos) >= len(ema_length):
                    continue
                sm = 0
                for dx in grid_pos:
                    for dy in grid_pos:
                        sm += ret[(dc_len[i+dx], ema_length[j+dy], st)]
                if sm > cur_best:
                    cur_best = sm
                    cur_params = (dc_l,ema_l)
        if (cur_best/16 < 0.05) and (cur_best/16 > 0.02) : best_val[st] = cur_params 
        return best_val if len(best_val) else {st: cur_params}

In [ ]:
start_date = '2023-03-01'
date_ret = {}
for i in range(9):
    end_date , next_start, next_end, nstart_date = get_last_date(start_date)


    ohlcv = vbt.YFData.download(stockss, start=start_date, end=end_date, interval='1h')
    print(f'running optimization from {start_date} to {end_date}')
    start_date = nstart_date

    ema_length=np.arange(3,25,2)
    dc_len=np.arange(7,27,1)
    signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_length, dc_len=dc_len)
    entries = signal.trends_crossed_above(0)
    exits = signal.trends_crossed_below(0)

    pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
    best_val = optimize(pf,stockss,dc_len,ema_length) 
    num_st = len(best_val) 
    print(f"num of stocks sorted for testing {num_st} ")
    print(f"testing optimazation from f{next_start} to {next_end}")                                                                         
    future_ret = {}
    future_ret2={}
    for key, val in best_val.items():
        dc_len , ema_l = val
        
        ohlcv = vbt.YFData.download(key, start=next_start, end=next_end, interval='1h')
        signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_l, dc_len=dc_len)
        entries = signal.trends_crossed_above(0)
        exits = signal.trends_crossed_below(0)

        pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
        future_ret[key] = pf.total_return()
        dc_len -= 2 if dc_len > 3 else dc_len
        ema_l -= 2 if ema_l > 3 else ema_l
        signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_l, dc_len=dc_len)
        entries = signal.trends_crossed_above(0)
        exits = signal.trends_crossed_below(0)

        pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
        future_ret2[key] = pf.total_return()
    # print(future_ret)
    date_ret[next_end] = future_ret
    print('cummulative return ', sum(future_ret.values())/num_st)
    print('cummulative return with -2 ', sum(future_ret2.values())/num_st)

    








In [27]:
print(len(entries[50:]))
ohlcv.get("Close").iloc[50:]


445


symbol,RELIANCE.NS,TCS.NS,HDFCBANK.NS,ICICIBANK.NS,INFY.NS,SBIN.NS,BHARTIARTL.NS,LICI.NS,HINDUNILVR.NS,ITC.NS,...,CIPLA.NS,INDUSINDBK.NS,AMBUJACEM.NS,HINDALCO.NS,ATGL.NS,TATACONSUM.NS,EICHERMOT.NS,CANBK.NS,DRREDDY.NS,TVSMOTOR.NS
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-03 18:30:00+00:00,2469.429199,3312.681641,1325.217529,714.052124,1444.710938,463.276886,715.914795,NaN,2086.799316,244.732056,...,945.254272,959.946899,363.851410,454.486542,2445.878174,788.207214,2439.853271,207.336945,3908.176758,624.419006
2022-05-04 18:30:00+00:00,2421.070068,3344.716309,1322.236694,718.538025,1491.376709,463.614960,712.495789,NaN,2082.042480,246.899063,...,936.221741,919.225464,362.121613,454.437347,2449.326660,761.763611,2391.069824,209.057785,3880.228516,627.951599
2022-05-05 18:30:00+00:00,2402.505859,3267.795654,1287.689087,709.122559,1451.578979,467.430145,703.873901,NaN,2065.321289,251.233124,...,922.747070,898.226990,361.182556,436.864319,2436.482666,750.577820,2321.211182,204.034821,3847.777344,625.961365
2022-05-08 18:30:00+00:00,2308.675537,3279.696045,1289.888062,700.347839,1477.451904,459.654907,705.112610,NaN,2034.041626,247.747055,...,926.251404,873.009094,362.516968,437.208893,2374.112549,744.003662,2303.377930,187.291595,3823.451416,612.428223
2022-05-09 18:30:00+00:00,2268.658936,3273.650879,1310.606934,701.284485,1461.128540,458.978821,713.189453,NaN,2095.303711,244.732056,...,914.257507,893.664185,354.065674,416.436279,2359.519287,731.787598,2373.384277,185.756790,3831.478516,613.174500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-11 18:30:00+00:00,2904.699951,4120.700195,1390.000000,996.099976,1680.099976,707.900024,1118.699951,1022.450012,2385.649902,406.850006,...,1434.750000,1447.150024,563.500000,582.450012,1025.250000,1122.199951,3871.850098,545.750000,6333.500000,2035.750000
2024-02-12 18:30:00+00:00,2930.199951,4149.500000,1394.449951,1019.799988,1684.550049,713.450012,1117.849976,1011.450012,2394.000000,406.750000,...,1457.750000,1463.900024,568.250000,510.100006,1007.799988,1137.800049,3854.199951,552.200012,6335.350098,2044.250000
2024-02-13 18:30:00+00:00,2962.750000,4100.100098,1384.050049,1022.700012,1666.199951,743.400024,1115.849976,1070.500000,2387.949951,411.549988,...,1425.349976,1481.500000,570.900024,507.799988,1010.500000,1135.599976,3901.550049,566.450012,6258.350098,2087.949951


In [21]:
for fdic in date_ret.values():
    nd = [(k,v) for k,v in fdic.items() if v > 0.1]
    print(len(nd), nd)

6 [('ITC.NS', 0.11683898735590163), ('TATAMTRDVR.NS', 0.12005940174562565), ('ADANIGREEN.NS', 0.8334289472210445), ('PNB.NS', 0.10518666972276217), ('ATGL.NS', 0.21251348794220554), ('CANBK.NS', 0.13420351369931713)]
11 [('TATAMOTORS.NS', 0.2022506117323891), ('TATAMTRDVR.NS', 0.13878856556773056), ('BAJAJ-AUTO.NS', 0.10747494894863353), ('DLF.NS', 0.1332460929628719), ('LTIM.NS', 0.1542839997717124), ('PNB.NS', 0.11232209534139183), ('ZOMATO.NS', 0.10036463752537733), ('TATAPOWER.NS', 0.14326475743030082), ('TATACONSUM.NS', 0.11365958297713252), ('EICHERMOT.NS', 0.12721261007596563), ('TVSMOTOR.NS', 0.16604569479261524)]
12 [('ADANIENT.NS', 0.33531292201262375), ('TATAMOTORS.NS', 0.10269986973301988), ('TATAMTRDVR.NS', 0.1751220282248949), ('TITAN.NS', 0.1145306435750032), ('DMART.NS', 0.15991574171354675), ('M&M.NS', 0.10119910845152932), ('HAL.NS', 0.1986543560381449), ('HDFCLIFE.NS', 0.1301924692528794), ('INDIGO.NS', 0.13500132724858763), ('INDUSINDBK.NS', 0.1526983442241157), ('A

In [71]:
## testing 
start_date = '2023-12-10'
end_date = '2024-02-15'
# ohlcv = vbt.YFData.download(stockss, start=start_date, end=end_date, interval='1h')



In [83]:
future_ret = {}
future_pf = {}
future_signal= {}
for key, val in best_val.items():
    dc_len , ema_l = val
    dc_len -= 1
    ema_l -= 1
    ohlcv = vbt.YFData.download(key, start=start_date, end=end_date, interval='1h')
    signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_l, dc_len=dc_len)
    entries = signal.trends_crossed_above(0)
    exits = signal.trends_crossed_below(0)

    pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
    future_ret[key] = pf.total_return()
    future_pf[key] = pf
    future_signal[key] = signal
print(future_ret)
print(sum(future_ret.values()))

{'RELIANCE.NS': 0.055704072101344065, 'TCS.NS': 0.10813381829339269, 'HDFCBANK.NS': -0.0416592029834482, 'ICICIBANK.NS': -0.03926429477566009, 'INFY.NS': 0.03623704063186, 'SBIN.NS': 0.10975605559836171, 'BHARTIARTL.NS': 0.09925024117861933, 'LICI.NS': 0.20418687587524298, 'HINDUNILVR.NS': -0.0504678028286522, 'ITC.NS': 0.027080228535537463, 'LT.NS': -0.017836357862437435, 'HCLTECH.NS': 0.06918291856675758, 'BAJFINANCE.NS': 0.047366288808740935, 'ADANIENT.NS': 0.05980109118282562, 'SUNPHARMA.NS': 0.1379634906010938, 'MARUTI.NS': 0.03402344699534093, 'ONGC.NS': 0.18887096395708228, 'KOTAKBANK.NS': -0.01074435063854762, 'TATAMOTORS.NS': 0.22083881122889928, 'TATAMTRDVR.NS': 0.20467544722053863, 'AXISBANK.NS': -0.024077650368959147, 'NTPC.NS': 0.025971271927538474, 'TITAN.NS': -0.008228676327067806, 'ADANIGREEN.NS': 0.021692672949645348, 'COALINDIA.NS': 0.049227102658875664, 'ASIANPAINT.NS': -0.018981861885941326, 'ULTRACEMCO.NS': -0.0759438517209515, 'WIPRO.NS': 0.14122414741791756, 'ADA

In [ ]:
for key in future_signal:
    plot_orders(future_pf[key],future_signal[key],key)

In [ ]:
for k , pf in future_pf.items():
    print(k)
    pf.plot_trade_pnl().show()

In [50]:
sum(future_ret.values())

1.9532574366555797

In [52]:
start_date = '2023-10-01'
end_date = '2023-12-31'

In [53]:
future_ret = {}
future_stat = {}
for key, val in best_val.items():
    dc_len , ema_l = val
    ohlcv = vbt.YFData.download(key, start=start_date, end=end_date, interval='1h')
    signal = madc.run(ohlcv.get("Close"), ohlcv.get("Low"), ohlcv.get("High"), ema_length=ema_l, dc_len=dc_len)
    entries = signal.trends_crossed_above(0)
    exits = signal.trends_crossed_below(0)

    pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
    future_ret[key] = pf.total_return()
    future_stat[key] = pf.stats()
print(future_ret)
print(sum(future_ret.values()))

{'RELIANCE.NS': 0.08959260378591556, 'TCS.NS': 0.10424617032937589, 'HDFCBANK.NS': 0.10258635999573643, 'ICICIBANK.NS': 0.1001855252057359, 'INFY.NS': 0.1406736249440073, 'SBIN.NS': 0.1200754736788175, 'BHARTIARTL.NS': 0.10598389519937046, 'LICI.NS': 0.2539094357058972, 'HINDUNILVR.NS': 0.0786946146808454, 'ITC.NS': 0.057325064349739115, 'LT.NS': 0.16732582959628198, 'HCLTECH.NS': 0.12570765796691719, 'BAJFINANCE.NS': 0.03758663801481846, 'ADANIENT.NS': 0.2384990043571555, 'SUNPHARMA.NS': 0.12580153315850867, 'MARUTI.NS': 0.03205049750519145, 'ONGC.NS': 0.12488967836988202, 'KOTAKBANK.NS': 0.04277888607574496, 'TATAMOTORS.NS': 0.1981679440459142, 'TATAMTRDVR.NS': 0.14886299927522967, 'AXISBANK.NS': 0.09579155418395924, 'NTPC.NS': 0.2561815116881447, 'TITAN.NS': 0.13522039949670622, 'ADANIGREEN.NS': 0.8257099525942577, 'COALINDIA.NS': 0.19674298793180292, 'ASIANPAINT.NS': 0.10881776779491832, 'ULTRACEMCO.NS': 0.21803878259945284, 'WIPRO.NS': 0.115420067212945, 'ADANIPORTS.NS': 0.2853167

symbol,RELIANCE.NS,TCS.NS,HDFCBANK.NS,ICICIBANK.NS,INFY.NS,SBIN.NS,BHARTIARTL.NS,LICI.NS,HINDUNILVR.NS,ITC.NS,...,CIPLA.NS,INDUSINDBK.NS,AMBUJACEM.NS,HINDALCO.NS,ATGL.NS,TATACONSUM.NS,EICHERMOT.NS,CANBK.NS,DRREDDY.NS,TVSMOTOR.NS
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-03 03:45:00+00:00,2323.300049,3482.300049,1511.000000,944.750000,1425.449951,593.650024,920.299988,645.599976,2491.000000,440.899994,...,1177.650024,1434.000000,428.049988,478.299988,613.150024,872.150024,3326.699951,379.000000,5504.000000,1530.050049
2023-10-03 04:45:00+00:00,2322.699951,3502.100098,1510.699951,943.299988,1424.050049,593.400024,921.549988,645.400024,2496.000000,441.250000,...,1183.900024,1434.449951,430.100006,479.899994,610.799988,871.599976,3328.949951,376.600006,5497.000000,1524.699951
2023-10-03 05:45:00+00:00,2323.449951,3509.750000,1514.050049,942.750000,1429.000000,595.000000,923.500000,645.900024,2495.600098,440.899994,...,1184.150024,1433.349976,431.350006,479.299988,611.849976,868.049988,3341.600098,378.149994,5482.000000,1526.050049
2023-10-03 06:45:00+00:00,2325.100098,3524.949951,1513.050049,942.049988,1432.949951,596.200012,923.650024,645.549988,2493.350098,441.450012,...,1185.000000,1433.250000,431.450012,479.649994,612.000000,871.450012,3340.699951,379.200012,5472.799805,1523.699951
2023-10-03 07:45:00+00:00,2319.199951,3502.649902,1509.750000,939.549988,1430.699951,599.049988,926.849976,645.400024,2476.199951,440.049988,...,1180.300049,1436.050049,433.850006,479.549988,610.700012,867.650024,3342.000000,381.100006,5468.700195,1525.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29 05:45:00+00:00,2595.000000,3802.300049,1710.300049,1000.400024,1545.000000,642.150024,1034.699951,828.599976,2656.750000,463.899994,...,1246.449951,1595.900024,520.750000,610.349976,990.250000,1078.699951,4116.000000,439.200012,5801.049805,2008.750000
2023-12-29 06:45:00+00:00,2593.899902,3800.050049,1711.750000,997.799988,1543.449951,641.849976,1034.099976,829.349976,2659.550049,463.500000,...,1244.300049,1594.150024,518.549988,616.400024,989.400024,1076.800049,4117.000000,439.649994,5779.750000,2005.150024
2023-12-29 07:45:00+00:00,2585.550049,3796.350098,1706.449951,994.750000,1544.500000,640.250000,1030.800049,827.000000,2656.649902,464.149994,...,1243.250000,1588.400024,516.500000,610.599976,984.400024,1077.500000,4116.250000,436.750000,5790.200195,2015.500000


## newsection

In [64]:
ret = pf.benchmark_returns()
ret

ma_dc_len                          9                                      \
ma_ema_length                      2                                       
symbol                    RELIANCE.NS    TCS.NS HDFCBANK.NS ICICIBANK.NS   
Date                                                                       
2020-02-16 18:30:00+00:00    0.000000  0.000000    0.000000     0.000000   
2020-02-17 18:30:00+00:00   -0.007340  0.005673   -0.003204    -0.000738   
2020-02-18 18:30:00+00:00    0.024806 -0.008756    0.011498     0.006652   
2020-02-19 18:30:00+00:00   -0.011870 -0.018007   -0.008230     0.004038   
2020-02-23 18:30:00+00:00   -0.027592 -0.019218   -0.005874    -0.031353   
...                               ...       ...         ...          ...   
2024-02-08 18:30:00+00:00    0.007327 -0.000097    0.000392     0.021631   
2024-02-11 18:30:00+00:00   -0.005750 -0.003145   -0.009689    -0.014445   
2024-02-12 18:30:00+00:00    0.008779  0.006989    0.003201     0.023793   
2024-02-13 18:30:00+00:00    0.011108 -0.011905   -0.007458     0.002844   
2024-02-14 18:30:00+00:00   -0.007274  0.000756    0.021676    -0.005916   

ma_dc_len                                                              \
ma_ema_length                                                           
symbol                      INFY.NS   SBIN.NS   LICI.NS BHARTIARTL.NS   
Date                                                                    
2020-02-16 18:30:00+00:00  0.000000  0.000000  0.000000      0.000000   
2020-02-17 18:30:00+00:00  0.010387  0.010662  0.000000     -0.029388   
2020-02-18 18:30:00+00:00  0.003510  0.008818  0.000000     -0.008846   
2020-02-19 18:30:00+00:00 -0.004310  0.022788  0.000000      0.004049   
2020-02-23 18:30:00+00:00 -0.001443 -0.014345  0.000000     -0.028776   
...                             ...       ...       ...           ...   
2024-02-08 18:30:00+00:00 -0.013445  0.036738 -0.022076     -0.019174   
2024-02-11 18:30:00+00:00  0.006440 -0.023923 -0.054032     -0.001384   
2024-02-12 18:30:00+00:00  0.002649  0.007840 -0.010758     -0.000760   
2024-02-13 18:30:00+00:00 -0.010893  0.041979  0.058382     -0.001789   
2024-02-14 18:30:00+00:00  0.006092  0.024617 -0.013452      0.004346   

ma_dc_len                                          ...           20            \
ma_ema_length                                      ...           15             
symbol                    HINDUNILVR.NS    ITC.NS  ... TATASTEEL.NS   LTIM.NS   
Date                                               ...                          
2020-02-16 18:30:00+00:00      0.000000  0.000000  ...     0.000000  0.000000   
2020-02-17 18:30:00+00:00     -0.008080 -0.004639  ...    -0.012476 -0.014165   
2020-02-18 18:30:00+00:00      0.025846  0.013981  ...     0.004172  0.001681   
2020-02-19 18:30:00+00:00     -0.019152  0.003628  ...     0.023892  0.020372   
2020-02-23 18:30:00+00:00     -0.014723 -0.021451  ...    -0.063353 -0.003340   
...                                 ...       ...  ...          ...       ...   
2024-02-08 18:30:00+00:00      0.002937  0.002292  ...    -0.016359  0.005995   
2024-02-11 18:30:00+00:00     -0.015882 -0.020818  ...    -0.027601  0.011589   
2024-02-12 18:30:00+00:00      0.003500 -0.000246  ...     0.003275  0.007369   
2024-02-13 18:30:00+00:00     -0.002527  0.011801  ...     0.024302 -0.009357   
2024-02-14 18:30:00+00:00     -0.015536 -0.018588  ...     0.000000  0.003115   

ma_dc_len                                                                \
ma_ema_length                                                             
symbol                    SIEMENS.NS SBILIFE.NS BANKBARODA.NS    PNB.NS   
Date                                                                      
2020-02-16 18:30:00+00:00   0.000000   0.000000      0.000000  0.000000   
2020-02-17 18:30:00+00:00   0.008208   0.001804     -0.015796  0.000962   
2020-02-18 18:30:00+00:00   0.019941   0.024180      0.001235  0.013462   
2020-02-19 18:30:00+00

In [28]:
returns

ma_dc_len  ma_ema_length  symbol      
5          3              RELIANCE.NS     0.910585
                          TCS.NS          0.079753
                          HDFCBANK.NS     0.014316
                          ICICIBANK.NS    0.497085
                          INFY.NS         0.957298
                                            ...   
14         10             PNB.NS          2.817464
                          GRASIM.NS       1.429876
                          BPCL.NS         0.332803
                          TRENT.NS        2.810862
                          BEL.NS          2.418032
Name: total_return, Length: 150, dtype: float64

In [ ]:
def plot_order_with_data(key):

In [56]:
signal = zsignal.run(ohlcv.get("Close"), length=70,lower_threshold=-2.5, upper_threshold=3.3,param_product=True)
entries = signal.signal == 1
exits = signal.signal == -1

pf = vbt.Portfolio.from_signals(ohlcv.get("Close"), entries=entries,exits=exits,freq='1h', init_cash=100000,cash_sharing=False)
pf.stats()


(3446, 50)


Start                         2022-02-15 03:45:00+00:00
End                           2024-02-14 09:45:00+00:00
Period                                143 days 14:00:00
Start Value                                    100000.0
End Value                                 215746.109552
Total Return [%]                              115.74611
Benchmark Return [%]                          83.655851
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              28.702515
Max Drawdown Duration                 103 days 02:00:00
Total Trades                                          6
Total Closed Trades                                   5
Total Open Trades                                     1
Open Trade PnL                            109156.848968
Win Rate [%]                                       80.0
Best Trade [%]                                18.793389
Worst Trade [%]                              -20

In [43]:
# returns = pf.total_return()
# print(returns.to_string())
pf[(70,-2.5,3.3,"TCS.NS")].plot_orders()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '22720e14-7181-4e61-b966-61f358424d78',
              'x': array([datetime.datetime(2022, 2, 15, 3, 45, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 2, 15, 4, 45, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 2, 15, 5, 45, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 2, 14, 7, 45, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 2, 14, 8, 45, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 2, 14, 9, 45, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([3754.39990234, 3760.94995117, 3786.25      , ..., 4100.        ,
                          4090.5       , 4100.100

In [32]:
returns.xs(key=80, level="ma_length")

ma_lower_threshold  ma_upper_threshold  symbol       
-1.5                2.8                 ABB.NS           0.729327
                                        ADANIENSOL.NS    0.086604
                                        ADANIENT.NS      1.128867
                                        ADANIGREEN.NS   -0.027758
                                        ADANIPORTS.NS    0.415269
                                                           ...   
-3.1                3.3                 VBL.NS           0.163346
                                        VEDL.NS         -0.303232
                                        WIPRO.NS        -0.282549
                                        ZOMATO.NS        0.286834
                                        ZYDUSLIFE.NS     0.279760
Name: total_return, Length: 909, dtype: float64

In [13]:
# returns.xs(key=80, level="ma_length")
fig = returns.vbt.volume(
    x_level= "ma_lower_threshold",
    y_level= "ma_upper_threshold",
    z_level="ma_length",
   slider_level = "symbol"
)
fig.show()
# for stock in stock_list:
#     pf[(80, -1.0, 3.3, stock)].plot_trade_pnl().show()

In [50]:
order = pf['ITC.NS'].get_orders()
pd.DataFrame(order)

ValueError: DataFrame constructor not properly called!

In [ ]:
# print(pf.total_return())
for stock in stock_list:
    print(stock)
    pf[stock].plot().show()

In [54]:
pf.plot_trade_pnl()

FigureWidget({
    'data': [{'customdata': array([[0.00000000e+00, 0.00000000e+00, 1.16368111e+03, 1.16368111e-02],
                                   [4.00000000e+00, 4.00000000e+00, 4.46102118e+03, 4.53465085e-02],
                                   [5.00000000e+00, 5.00000000e+00, 1.36222995e+04, 1.32464566e-01],
                                   [6.00000000e+00, 6.00000000e+00, 5.14794190e+03, 4.42036681e-02]]),
              'hovertemplate': ('Exit Trade Id: %{customdata[0]' ... 'r>Return: %{customdata[3]:.2%}'),
              'marker': {'color': '#37B13F',
                         'line': {'color': 'rgb(38,123,44)', 'width': 1},
                         'opacity': array([0.75436894, 0.7949985 , 0.9       , 0.79362106]),
                         'size': array([ 7.20388383,  9.09993015, 14.        ,  9.03564959]),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Closed - Profit',
              'type': 'scatter',
              'u

# temp

In [107]:
help(ta.ema)

Help on function ema in module pandas_ta.overlap.ema:

ema(close, length=None, talib=None, offset=None, **kwargs)
    Exponential Moving Average (EMA)
    
    The Exponential Moving Average is more responsive moving average compared to the
    Simple Moving Average (SMA).  The weights are determined by alpha which is
    proportional to it's length.  There are several different methods of calculating
    EMA.  One method uses just the standard definition of EMA and another uses the
    SMA to generate the initial value for the rest of the calculation.
    
    Sources:
        https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages
        https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
    
    Calculation:
        Default Inputs:
            length=10, adjust=False, sma=True
        if sma:
            sma_nth = close[0:length].sum() / length
            close[:length - 1] = n

In [109]:
help(ta.cross_value)

Help on function cross_value in module pandas_ta.utils._signals:

cross_value(series_a: pandas.core.series.Series, value: float, above: bool = True, asint: bool = True, offset: int = None, **kwargs)

